# Initial Scraping

I'll use this notebook to understand the Total Wine Website and build my scraper. I also had to use some selenium to click some "expand" and "I'm 21" buttons.

I'll also outline the my plans for this project along the way.


Goal : Create a Scotch Recommender

Details: Utilize subjective information like text reviews containing flavor notes in combinations with qualitative information like bottle price and years aged.

Method: Scrape total wines website for all scotch reviews.  
    - In this process i'll probably figure out how to scrape all other liquors and their reviews so I can repeat the process in the future for things such as bourbon or tequila. 
    - I also plan on utilizing a local Postgres SQL server to house the information.

I did a lot of experimenting in this notebook and when I came back things didn't work quite right.  I'll probably need to investigate it again.

In [7]:
# Step 1: Figure our how to scrape a single page
from bs4 import BeautifulSoup
import requests
import re
import urllib

import pandas as pd
from IPython.display import clear_output


from selenium import webdriver
from scrapy.selector import Selector
import time
import csv
from itertools import islice

In [2]:
# Test against a single scotch page.
scotch = 'https://www.totalwine.com/spirits/scotch/single-malt/lagavulin-16-yr/p/643750?s=2301&igrules=true'
requests.get(scotch)

<Response [200]>

_Looks Good_

In [3]:
# test against a single tequila page.
tequila = 'https://www.totalwine.com/spirits/tequila/blanco-silver/don-julio-1942-tequila/p/38388750?s=2301&igrules=true'

In [4]:
sco_soup = BeautifulSoup(requests.get(scotch).content, "html5lib")
teq_soup = BeautifulSoup(requests.get(tequila).content, 'lxml')

In [5]:
def spirit_extractor(sco_soup):

    # Item Info
    title = sco_soup.find('h1', {'class':'product-name'}).find('input')['value']
    print(title)

    price = sco_soup.find('input', {'id':"anPRice"})['value']
    print(price)

    size = sco_soup.find('span', {'class':"single-bottle bottle-options"}).text
    print(size)

    avg_rating = sco_soup.find('span', {'class':"bvseo-ratingValue"}).text
    print(avg_rating)

    # Item Overview
    overview = sco_soup.find(name = 'div', attrs = {'id':"product-details-section"})

    overview_taste = overview.find('div' , {'class':"bottom-full-desc"}).text
    print(overview_taste)

    overview_score = overview.find('span', {'class':'nine-six-logo'}).text
    print(overview_score)

    overview_desc = overview.find('div', {'class':'right-full-desc'}).text
    print(overview_desc)

    brand = sco_soup.find('main', {'class': "main pdp an-productDetails"}).find('meta')['content']
    print(brand)
    # Item Details
    details = sco_soup.find('ul', {'class' : "wine-right-details"})

    country = details.find('a',{'class':'analyticsCountryState'}).text
    print(country)

    state = details.find('a',{'class':'analyticsRegion'}).text
    print(state)

    spirit_type = details.find('a',{'class':'analyticsProductType'}).text
    print(spirit_type)

    spirit_style = details.find('a',{'class':'analyticsVarietalType'}).text
    print(spirit_style)



In [ ]:
# Skew

In [6]:
reviews = sco_soup.find('ol', )
reviews

<ol itemscope="" itemtype="http://schema.org/BreadcrumbList"> <li itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem"> <a href="https://www.totalwine.com" itemprop="item"> <meta content="Home" itemprop="name"/> </a> <meta content="1" itemprop="position"/> </li> <li itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem">	<a href="https://www.totalwine.com/spirits/c/c0030" itemprop="item">	<meta content="Spirits" itemprop="name"/>	</a>	<meta content="2" itemprop="position"/>	</li>	<li itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem">	<a href="https://www.totalwine.com/spirits/scotch/c/000885" itemprop="item">	<meta content="Scotch" itemprop="name"/>	</a>	<meta content="3" itemprop="position"/>	</li>	<li itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem">	<a href="https://www.totalwine.com/spirits/scotch/single-malt/c/000887" itemprop="item">	<meta content="Single Malt" itemprop="name"

In [8]:
def connect_to_page(page,  headless=True):
    if headless:
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        driver = webdriver.Chrome('./driver/chromedriver', chrome_options=options)
    else:
        driver = webdriver.Chrome('./driver/chromedriver')
    driver.get(page)

    return driver

**Issues**
- Selenium opens up a new browser with no cookies
- Need to make some selections once it opens up in order to have access to the website.
- Need to create a while loop to figure out how many times it should load additional comments.
    - I think I'll get the number of comments off the page then use that.

In [9]:
# couple of various sctoches to test on.
lagavulin_16 = 'https://www.totalwine.com/spirits/scotch/single-malt/lagavulin-16-yr/p/643750?'
glenlevit_12 = 'https://www.totalwine.com/spirits/scotch/single-malt/glenlivet-12-yr/p/2792010?s=2301&igrules=true'
jonnieW_gold = 'https://www.totalwine.com/spirits/scotch/blended-scotch/johnnie-walker-gold-label-reserve/p/137781750?s=2301&igrules=true'


samples = [lagavulin_16, glenlevit_12, jonnieW_gold]

This is the base page I'll use to gather all the scotch urls.

https://www.totalwine.com/spirits/scotch/c/000885?tab=fullcatalog&text=&page=1&spiritsvolume=standard-size-750-ml&sort=name-asc
    - `scotch` : Only Scotches
    - `fullcatalog` : All scotches available at total wine.
    - `volume` : Only aquiring standard bottle size, to avoid duplications based on different bottle volumes.

In [10]:
driver = connect_to_page(samples[0], headless = False)
load = False

# Total wine appears to have two different options that ask for age
while load == False:
    try:
        # Option 1 : Red Button
        red_age = '//*[@id="btnYes"]'
        driver.find_element_by_xpath(red_age).click()
        load = True
    except:
        pass

    try:
        # Option 2 : Blue Button
        blue_age = '/html/body/div[2]/div/div[2]/div[3]/div/button[2]'
        driver.find_element_by_xpath(blue_age).click()
        load = True
    except:
        pass

    time.sleep(0.5)


# if it prompts for online or instore via popup.
try:
    shop_button = '/html/body/header/div[3]/div/div[1]'
    driver.find_element_by_xpath(shop_button).click()

except:
    pass

# scroll down the page and click the "Load More" option as long as its available.
# load_more = True
# while load_more == True:
#     try:
#         load_button = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/button'
#         driver.find_element_by_xpath(load_button).click()
#         time.sleep(1)
#     except:
#         load_more = False

        
time.sleep(5)

html = driver.page_source

driver.get(samples[1])
time.sleep(10)

driver.get(samples[2])

time.sleep(10)
driver.quit()

In [19]:
def get_liquor_list(url, num_pages = 1, head_less = True):
    # get the driver going
    driver = connect_to_page(url, headless=head_less)
    time.sleep(1)
    load = False
    # Total wine appears to have two different options that ask for age
    while load == False:
        try:
            # Option 1 : Red Button
            red_age = '//*[@id="btnYes"]'
            driver.find_element_by_xpath(red_age).click()
            load = True
        except:
            pass

        try:
            # Option 2 : Blue Button
            blue_age = '/html/body/div[2]/div/div[2]/div[3]/div/button[2]'
            driver.find_element_by_xpath(blue_age).click()
            load = True
        except:
            pass

        time.sleep(0.5)
    
        # if it prompts for online or instore via popup.
    try:
        shop_button = '/html/body/header/div[3]/div/div[1]'
        driver.find_element_by_xpath(shop_button).click()

    except:
        pass
    
    # Now that all the page buttons have been solved, we can extract the URLs for each of the pages.
    # extract page to disect
    page = BeautifulSoup(driver.page_source, 'lxml')
    liquor_urls = []
    bottles = page.find('ul',{'class':'plp-list'})

    for bottle in bottles.find_all('li'):
        liquor_urls.append('https://www.totalwine.com'+bottle.find('a')['href'])
        #print(liquor_urls[-1])
    
    
    # if we need to go to additional pages.
    if num_pages > 1:
        for page in range(2,num_pages+1):
            page_add = url+'&page='+str(page)
            driver.get(page_add)
            page = BeautifulSoup(driver.page_source, 'lxml')
            bottles = page.find('ul',{'class':'plp-list'})

            for bottle in bottles.find_all('li'):
                liquor_urls.append('https://www.totalwine.com'+bottle.find('a')['href'])
            time.sleep(4)
    
    # End driver and return URLs list.
    driver.quit()
    return liquor_urls

In [14]:
# Calculating number of pages of scotches.  
1425/180
# approximately 8

7.916666666666667

In [20]:
# Connect to main scotch page
part_1 = 'https://www.totalwine.com/spirits/scotch/c/000885?tab=fullcatalog&text=&page='
part_2 = '&spiritsvolume=standard-size-750-ml&sort=name-asc&pagesize=180'

#scotch_home = 'https://www.totalwine.com/spirits/scotch/c/000885?tab=fullcatalog&text=&page={}&spiritsvolume=standard-size-750-ml&sort=name-asc&pagesize=180'
scotch_home = 'https://www.totalwine.com/spirits/scotch/c/000885?tab=fullcatalog&text=&spiritsvolume=standard-size-750-ml&pagesize=180'
#driver = connect_to_page(scotch_home, headless=False)
liquor_list = get_liquor_list(scotch_home, 8, False)







#page_html = get_and_expand_page()

In [22]:
#scotch_home_soup = BeautifulSoup(liquor_list, 'lxml')

In [23]:
# scotch_urls = []
# bottles = scotch_home_soup.find('ul',{'class':'plp-list'})

# for scotch in bottles.find_all('li'):
#     scotch_urls.append('https://www.totalwine.com'+scotch.find('a')['href'])
#     print(scotch.find('a')['href'])

NameError: name 'scotch_home_soup' is not defined

In [ ]:
driver.quit()

In [25]:
# Functiont to Aquire full page of Liquor.

def get_and_expand_page(page_url, driver = driver):
    driver.get(page_url)
    
    time.sleep(0.5) #wait to load
    
    
        # scroll down the page and click the "Load More" option as long as its available.
    load_more = True
    while load_more == True:
        try:
            load_button = '//*[@id="BVRRContainer"]/div/div/div/div/div[3]/div/button'
            driver.find_element_by_xpath(load_button).click()
            time.sleep(1)
        except:
            load_more = False
            
    return driver.page_source

In [ ]:
#soup_page = BeautifulSoup(html,'lxml')

In [35]:
def spirit_extractor(liquor_page):
    spirit_list = []
    # Spirit Info
    xpath_title = '//*[@id="root"]/div/div/div[2]/div/div[3]/div/div[2]/div/h1/text()'
    xpath_size  = '//*[@id="root"]/div/div/div[2]/div/div[3]/div/div[2]/div/h2/text()'
    xpath_avg_rating = '//*[@id="root"]/div/div/div[2]/div/div[3]/div/div[2]/div/div/div[1]/div/div/div[2]/button/text()'
    xpath_num_review = '//*[@id="root"]/div/div/div[2]/div/div[3]/div/div[2]/div/div/div[1]/div/div/div[3]/button/text()'
    
    # Spirit Overview
    xpath_brand   = '//*[@id="root"]/div/div/div[2]/div/div[4]/div/div/div[2]/div/div[2]/div[1]/div[2]/a/text()'
    xpath_country = '//*[@id="root"]/div/div/div[2]/div/div[4]/div/div/div[2]/div/div[2]/div[2]/div[2]/a/text()'
    xpath_state   = '//*[@id="root"]/div/div/div[2]/div/div[4]/div/div/div[2]/div/div[2]/div[3]/div[2]/a/text()'
    xpath_sType   = '//*[@id="root"]/div/div/div[2]/div/div[4]/div/div/div[2]/div/div[2]/div[4]/div[2]/a/text()'
    xpath_sStyle  = '//*[@id="root"]/div/div/div[2]/div/div[4]/div/div/div[2]/div/div[2]/div[5]/div[2]/a/text()'
    xpath_taste   = '//*[@id="root"]/div/div/div[2]/div/div[4]/div/div/div[2]/div/div[2]/div[6]/div[2]/text()'    
    xpath_score = '//*[@id="root"]/div/div/div[2]/div/div[4]/div/div/div[2]/div/div[1]/div[1]/div[1]/div/div/text()'
    xpath_descr = '//*[@id="root"]/div/div/div[2]/div/div[4]/div/div/div[2]/div/div[1]/div[2]/text()'
    
    
    title = Selector(text=liquor_page).xpath(xpath_title).extract()
    size  = Selector(text=liquor_page).xpath(xpath_size).extract()
    avg_rating  = Selector(text=liquor_page).xpath(xpath_avg_rating).extract()
    num_reviews = Selector(text=liquor_page).xpath(xpath_num_review).extract()    
    brand   = Selector(text=liquor_page).xpath(xpath_brand).extract()
    country = Selector(text=liquor_page).xpath(xpath_country).extract()
    state   = Selector(text=liquor_page).xpath(xpath_state).extract()
    sType   = Selector(text=liquor_page).xpath(xpath_sType).extract()
    sStyle  = Selector(text=liquor_page).xpath(xpath_sStyle).extract()
    taste   = Selector(text=liquor_page).xpath(xpath_taste).extract()
    score   = Selector(text=liquor_page).xpath(xpath_score).extract()
    descr   = Selector(text=liquor_page).xpath(xpath_descr).extract()
    
    
    for cat in [title, size, avg_rating, num_reviews, brand, country, state, sType, sStyle, taste, score, descr]:
        if len(cat) >0 :
            spirit_list.append(cat[0])
        else:
            spirit_list.append('NA')
    print(title)
    print(size)
    print(avg_rating)
    print(num_reviews)    
    print(brand)
    print(country)
    print(state)
    print(sType)
    print(sStyle)
    print(taste)
    print(score)
    print(descr)
    
    spirit_list.append(aquire_reviews(liquor_page))
    
    sample_frame.loc[len(sample_frame)] = spirit_list
spirit_extractor(html)

['LAGAVULIN 16 YR']
['750ml']
['4.6']
['(67)']
['Lagavulin']
['Scotland']
['Islay & Islands']
['Scotch']
['Single Malt']
['Intense, Smoke, Peat']
['94']
['Islay, Scotland- One of the great classics from Islay. Remarkably balanced, with very dry and assertive flavors offset by the sweetness of Sherry oak. Complex, full-bodied, smoky, peaty tastes combine with a salty tang of the sea in this elegant yet powerful Single Malt.']


In [36]:
# Aquire Review Function is broken

In [34]:
def aquire_reviews(liquor_page, no_reviews = 0):
    if no_reviews > 0:
        reviews_list = '//*[@id="BVRRContainer"]/div/div/div/div/ol'
        review_section = Selector(text=liquor_page.page_source).xpath(reviews_list).extract()


        soup_reviews = BeautifulSoup(review_section[0], 'lxml')


        reviews_dict = {}
        itera = 0

        for review in soup_reviews.find_all('div',
            {'class':'bv-content-item-author-profile-offset bv-content-item-author-profile-offset-on'})[:-1]:

            
            review_desc = review.find('h4', {'class':'bv-content-title'}).text
            review_cont = review.find('div',{'class':'bv-content-summary-body-text'}).text
            review_scor = review.find('span',{'class':'bv-off-screen'}).text[0]

            review = {'review_desc':review_desc,
                      'review_cont':review_cont,
                      'review_scor':review_scor}

            reviews_dict['review_'+str(len(reviews_dict)+1)] = review
        # I had this return statement tabbed over so it only got one.
        return(reviews_dict)

    else:
        return 'No Reviews'

In [32]:
# Sample Frame to append
liquor_cols = ['title','size','avg_rating', 'num_reviews','brand','country','state',
              'spirit_type','spirit_style','taste','score','descr','reviews']
sample_frame = pd.DataFrame(None, columns= liquor_cols)

In [ ]:
[][0]

### Getting information off of the Scotch Page

In [33]:
url = 'https://www.totalwine.com/spirits/scotch/c/000885?tab=fullcatalog&text='

BeautifulSoup(requests.get(url))

TypeError: object of type 'Response' has no len()